## Importing Libraries

In [1]:
import os

from delta import *
from delta.tables import *

from dotenv import load_dotenv

import pyspark
from pyspark.sql.types import *
from pyspark.sql import functions as F

## Loading Environment Variables

In [2]:
load_dotenv()

api_key = os.getenv("API_KEY")

## Create Spark Session

In [3]:
#  Create a spark session with Delta
builder = pyspark.sql.SparkSession.builder.appName("esports_tournaments_silver") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

# Create spark context
spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

24/02/21 14:54:08 WARN Utils: Your hostname, pitta resolves to a loopback address: 127.0.1.1; using 192.168.100.7 instead (on interface enp6s0)
24/02/21 14:54:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/pitta/anaconda3/envs/esports-pipeline/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/pitta/.ivy2/cache
The jars for the packages stored in: /home/pitta/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ee3ea78b-703f-4b10-bb1a-05652675aafb;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 673ms :: artifacts dl 64ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   | 

## Load Recent Tournaments Data

In [12]:
df = spark.read.parquet('../../data/bronze/esports_tournaments')
df.printSchema()
df.show()

root
 |-- 0: string (nullable = true)

+---------+
|        0|
+---------+
|ErrorCode|
|ErrorCode|
|ErrorCode|
+---------+



### Clean Data

In [ ]:
na_rows = df.count() - df.na.drop().count()
print(na_rows)

In [ ]:
duplicate_rows = df.count() - df.dropDuplicates().count()
print(duplicate_rows)

In [ ]:
df.describe().show()

In [ ]:
df = df.withColumn('StartDate', F.when((df['StartDate'] == '0202-05-07') & (df['GameId'] == 785), '2022-05-07').otherwise(df['StartDate']))

In [ ]:
df = df.na.drop()
df = df.dropDuplicates()

### Set Data Type

In [ ]:
df_prep = df.withColumn('EndDate', df['EndDate'].cast(DateType())) \
            .withColumn('GameId', df['GameId'].cast(IntegerType())) \
            .withColumn('Location', df['Location'].cast(StringType())) \
            .withColumn('StartDate', df['StartDate'].cast(DateType()))  \
            .withColumn('Teamplay', df['Teamplay'].cast(IntegerType())) \
            .withColumn('TotalUSDPrize', df['TotalUSDPrize'].cast(FloatType())) \
            .withColumn('TournamentId', df['TournamentId'].cast(IntegerType())) \
            .withColumn('TournamentName', df['TournamentName'].cast(StringType()))


### Save Data

In [ ]:
local_path = f"../../data/silver/esports_tournaments"
df.coalesce(1).write.format("delta").option("header", "true").mode("overwrite").save(local_path)

In [ ]:
got_df = spark.read.format('delta').load('../../data/silver/esports_tournaments')
got_df.show()

## Load Games Genre

In [ ]:
df = spark.read.parquet('/../../data/bronze/esports_games_genre.parquet')
df.printSchema()
df.show()

### Clean Data

In [ ]:
na_rows = df.count() - df.na.drop().count()
print(na_rows)

In [ ]:
duplicate_rows = df.count() - df.dropDuplicates().count()
print(duplicate_rows)

In [ ]:
df = df.na.drop()
df = df.dropDuplicates()

### Save Data

In [ ]:
local_path = f"../../data/silver/esports_games_genre"
df.coalesce(1).write.format("delta").option("header", "true").mode("overwrite").save(local_path)
df.show()

## Load Games Awarding Prize Money

In [ ]:
df = spark.read.parquet('../../data/bronze/esports_games_awarding_prize_money.parquet')
df.printSchema()
df.show()

### Clean Data

In [ ]:
na_rows = df.count() - df.na.drop().count()
print(na_rows)

In [ ]:
duplicate_rows = df.count() - df.dropDuplicates().count()
print(duplicate_rows)

In [ ]:
df = df.na.drop()
df = df.dropDuplicates()

### Set Data Types

In [ ]:
df_prep = df.withColumn('GameId', df['GameId'].cast(IntegerType())) \
            .withColumn('GameName', df['GameName'].cast(StringType())) \
            .withColumn('TotalPlayers', df['TotalPlayers'].cast(IntegerType())) \
            .withColumn('TotalTournaments', df['TotalTournaments'].cast(IntegerType()))  \
            .withColumn('TotalUSDPrize', df['TotalUSDPrize'].cast(FloatType())) \

### Save Data

In [ ]:
local_path = f"../../data/silver/esports_games_awarding_prize_money_test"
df_prep.coalesce(1).write.format("parquet").option("header", "true").mode("overwrite").save(local_path)
df_prep.show()